In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

from mp3id import mp3ID
from musicBase import musicBase
from musicFinder import musicFinder
from musicPath import pathBasics
from timeUtils import clock, elapsed
from discogsBase import discogs
from multiArtist import multiArtist
from time import sleep
from pandas import DataFrame, Series
from discogsBase import discogs
from ioUtils import getFile
from searchUtils import findDirs
from fileUtils import getDirBasics
from unicodedata import normalize
from fsUtils import moveDir, setDir
from masterdb import getArtistAlbumsDB, discConv
## Python Version
import sys
print("Python: {0}".format(sys.version))

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-01-26 19:32:50.297287


In [2]:
disc = discogs()

Saved Discog Directory /Volumes/Music/Discog is Available
Local Discog Directory /Users/tgadfort/Music/Discog is Available
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-db/metadata exists
/Volumes/Music/Discog/albums-db/metadata exists
/Volumes/Music/Discog/diagnostic exists
/Volumes/Music/Discog/db exists


In [17]:
print("Getting Master Artist DB File")
discdf = disc.getMasterSlimArtistDiscogsDB()
artists = list(discdf["Artist"])
print(len(artists))
discdf.head()
# 589630
# 610522
# 636524
# 669087
# 712336
# 712486
# 712629

Getting Master Artist DB File
Loading data from /Volumes/Music/Discog/db/MasterSlimArtistDB.p
  --> This file is 57.0MB.
Loading /Volumes/Music/Discog/db/MasterSlimArtistDB.p
712629


,Name,Artist,Num,DiscArtist
1000,Dave Clarke,Dave Clarke,NaN,Dave Clarke
1000500,Club Pulse,Club Pulse,NaN,Club Pulse
100200,Dike,Dike,NaN,Dike
1002000,Larry Stokes,Larry Stokes,NaN,Larry Stokes
1002600,Gidd Sanchez,Gidd Sanchez,NaN,Gidd Sanchez


In [30]:
artistAlbumsDB = getArtistAlbumsDB(disc, force=True)

Current Time is Sun Jan 26, 2020 19:45:29 for 
=================================== Creating Artist Albums DB ===================================
Loading ArtistID Data
Loading data from /Volumes/Music/Discog/db/ArtistIDToAlbumNames.p
  --> This file is 190.8MB.
Loading /Volumes/Music/Discog/db/ArtistIDToAlbumNames.p
Creating Pandas DataFrame for 712628 Artists
	Shape --> (712628, 1)
DataFrame Shape is (712628, 1)
Saving Master Artist Albums DB File
  --> This file is 375.0MB.
Current Time is Sun Jan 26, 2020 19:48:17 for Done with 
=================================== Creating Artist Albums DB ===================================
Process [{0}] took 2.8 minutes.


In [6]:
def getMusicData(key, artist):
    retval = discdf[discdf[key] == artist]
    if retval.shape[0] > 0:
        return retval
    else:
        return None
    
from difflib import SequenceMatcher

def getRowByIndex(pdf, idx):
    return pdf.loc[idx]

In [7]:
artistIDToName = discdf["DiscArtist"].to_dict()
artistNameToID = {}
print("Found {0} ID -> Name entries".format(len(artistIDToName)))
for artistID,artistName in artistIDToName.items():
    if artistNameToID.get(artistName) is None:
        artistNameToID[artistName] = []
    artistNameToID[artistName].append(artistID)
print("Found {0} Name -> ID entries".format(len(artistNameToID)))

Found 712628 ID -> Name entries
Found 654969 Name -> ID entries


In [8]:
#artistDB = disc.getArtistIDToNameData()
#artistNameToID = {v: k for k,v in artistDB.items()}
mulArts  = multiArtist(cutoff=0.9, discdata=artistNameToID, exact=False)

In [9]:
try:
    myMusicNameIDMap = getFile("musicDiscogsMap.p")
    print("Found {0} music <-> discogs maps".format(len(myMusicNameIDMap)))
except:
    myMusicNameIDMap = {}
    print("Could not load music <-> discogs map")
       
myMusicNameIDMap["Mayday!"] == ['2156710', '¡Mayday!']

Found 4688 music <-> discogs maps


True

# Fix Discogs Mapping

In [10]:
def fixDiscogsMapping(myMusicNameIDMap):
    musicNameIDMap = {}
    for artistName,v in myMusicNameIDMap.items():
        if isinstance(v, list):
            musicNameIDMap[artistName] = v
            #print(artistName,v)
            continue

        mdata = getMusicData("Name", v)
        if isinstance(mdata, DataFrame):
            if mdata.shape[0] == 1:
                #print(artistName,[mdata.index[0], mdata["Name"].values[0]])
                musicNameIDMap[artistName] = [mdata.index[0], mdata["Name"].values[0]]
            else:
                raise ValueError("No idea about {0}".format(mdata))
        else:
            print(artistName,'\t',v)
            
    return musicNameIDMap
        
#myMusicNameIDMap = musicNameIDMap
#from ioUtils import saveFile
#saveFile(ifile="musicDiscogsMap.p", idata=musicNameIDMap, debug=True)

In [11]:
myMusicNameIDMap = fixDiscogsMapping(myMusicNameIDMap)

In [ ]:
if myMusicNameIDMap["Mayday!"] == ['2156710', '¡Mayday!']:
    saveFile(ifile="musicDiscogsMap.p", idata=myMusicNameIDMap, debug=True)
    saveFile(ifile="musicDiscogsMap.yaml", idata=myMusicNameIDMap, debug=True)

In [ ]:
manualMatches = {}

if len(manualMatches) > 0:
    myMusicNameIDMap.update(manualMatches) 

print("Found {0} music <-> discogs maps".format(len(myMusicNameIDMap)))

# Find Music and Match Artists

In [ ]:
rms = ["Billy Gibbons & The Blue Union"]
rms += ["C.B. Milton"]
rms += ["David Parmley & Continental Divide"]
rms += ["Hall And Oates"]
rms += ["John Trubee & The Ugly Janitors Of America"]
rms += ["Johnny Cash & Jerry Lee Lewis"]
rms += ["Les Beatniks De Queens"]
rms += ["Meathook Seed"]
rms += ["Snake Hips"]
rms += ["Steve Almaas & Ali Smith"]
rms += ["Varios (Band)"]
for rm in rms:
    del myMusicNameIDMap[rm]

In [18]:
musicNameIDMap = {}
multiMap       = {}

discogMediaNames = ['Albums', 'Singles & EPs', 'Compilations', 'Videos', 'Miscellaneous', 'Visual', 'DJ Mixes']
myMediaNames     = ['Random', 'Todo', 'Match', 'Title', 'Singles']

for dirN in findDirs("/Users/tgadfort/matched"):
    print(dirN)
    for dirval in findDirs(dirN):   
        artistName = getDirBasics(dirval)[-1]
        artistName = normalize('NFC', artistName)
        
        if myMusicNameIDMap.get(artistName) is not None:
            idx = myMusicNameIDMap[artistName][0]
            try:
                discogsArtistAlbumsData  = getRowByIndex(artistAlbumsDB, idx)
                discogsArtistAlbums      = discogsArtistAlbumsData["Albums"]
            except:
                print(idx,'\t',artistName)
            continue

        mdata = getMusicData("DiscArtist", artistName)
        print(artistName,type(mdata))
        if not isinstance(mdata, DataFrame):
            matches = mulArts.getArtistNames(artistName)
            if not all(matches.values()):
                print("{0: <10}{1: <50}{2}".format("?", artistName, matches))
                continue
            else:
                print("{0: <10}{1: <50}{2}".format("MIX", artistName, matches))
                srcDir = dirval
                dstDir = setDir("/Volumes/Music/Multi", artistName)
                moveDir(srcDir, dstDir)                
        else:
            print('\t',mdata.shape)
            if mdata.shape[0] == 1:
                musicNameIDMap[artistName] = [mdata.index, mdata["Name"].values[0]]
            else:
                print("{0: <10}{1: <50}{2}".format("", artistName, ""))
                albums = [x for x in findDirs(dirval) if getDirBasics(x)[-1] not in discogMediaNames+myMediaNames]
                if len(albums) == 0:
                    print("\tNo Albums\n")
                    srcDir = dirval
                    dstDir = setDir("/Volumes/Music/Random", artistName)
                    moveDir(srcDir, dstDir)
                else:
                    multiMap[artistName] = {"DB": mdata, "Albums": albums}
                    continue
                    for album in albums:
                        print("\t",album)
                    print("")
                print("\tDiscogs Data:")
                print(mdata)
                print("\n\n")

/Users/tgadfort/matched/A
/Users/tgadfort/matched/B
/Users/tgadfort/matched/C
/Users/tgadfort/matched/D
/Users/tgadfort/matched/E
/Users/tgadfort/matched/F
/Users/tgadfort/matched/G
/Users/tgadfort/matched/H
/Users/tgadfort/matched/I
/Users/tgadfort/matched/J
/Users/tgadfort/matched/K
/Users/tgadfort/matched/L
/Users/tgadfort/matched/M
Meathook Seed <class 'pandas.core.frame.DataFrame'>
	 (1, 4)
/Users/tgadfort/matched/N
/Users/tgadfort/matched/Num
/Users/tgadfort/matched/O
/Users/tgadfort/matched/P
/Users/tgadfort/matched/Q
/Users/tgadfort/matched/R
/Users/tgadfort/matched/S
/Users/tgadfort/matched/T
/Users/tgadfort/matched/The
/Users/tgadfort/matched/U
/Users/tgadfort/matched/V
/Users/tgadfort/matched/W
/Users/tgadfort/matched/Xtra
/Users/tgadfort/matched/Y
/Users/tgadfort/matched/Z


# Manual Entries

In [ ]:
newManual = {}

In [ ]:
mdata = getMusicData("Name", "Daryl Hall & John Oates")
newManual[mdata["DiscArtist"].values[0]] = [mdata.index, mdata["Name"].values[0]]

In [ ]:
newManual

In [ ]:
if len(newManual) > 0:
    try:
        myMusicNameIDMap.update({k: [v[0][0], v[1].values[0]] for k,v in newManual.items()})
    except:
        try:
            myMusicNameIDMap.update({k: [v[0][0], v[1]] for k,v in newManual.items()})
        except:
            raise ValueError("Not sure about {0}".format(newManual))

## Find MultiMap -> Match Entry

In [ ]:
ratVal = 0.6
if len(myMusicNameIDMap) > 0:
    ratVal = 0.3
artistIDMap = {}
for artistName,artistSlimData in multiMap.items():
    print("\n","="*50)
    print("ArtistName: {0}".format(artistName))
    print("   Albums: {0}".format(len(artistSlimData["Albums"])))

    myAlbumNames = []
    for album in artistSlimData["Albums"]:
        myAlbumName = album.split("/{0}/".format(artistName))[-1]
        #print("     {0: <15}{1: <10}{2}".format("", "", myAlbumName))
        myAlbumNames.append(myAlbumName)
        
    for idx,row in artistSlimData["DB"].iterrows():
        artistAlbumsData = getRowByIndex(artistAlbumsDB, idx)
        artistAlbums     = artistAlbumsData["Albums"]
        print("   Match: {0}  [{1}]".format(row["DiscArtist"], row["Name"]))
        if isinstance(artistAlbums, dict):
            for albumType, albumTypeData in artistAlbums.items():
                for albumID, dbAlbumName in albumTypeData.items():
                    for myAlbumName in myAlbumNames:
                        s = SequenceMatcher(None, myAlbumName, dbAlbumName)
                        ratio = s.ratio()
                        if ratio > 0.6:
                            print("     {0: <15}{1: <10}{2: <8}{3}".format(albumType, albumID, round(ratio,2), dbAlbumName))
                            artistIDMap[artistName] = [idx, row["Name"]]

## Found Multi Artist

In [ ]:
for artistName,artistData in artistIDMap.items():
    print(artistName,' ==> ',artistData)

In [ ]:
if len(artistIDMap) > 0:
    myMusicNameIDMap.update(artistIDMap)

## Set Single Artist

In [19]:
for artistName,artistData in musicNameIDMap.items():
    print(artistName,' ==> ',artistData)

Meathook Seed  ==>  [Index(['300089'], dtype='object'), 'Meathook Seed']


In [20]:
print("Found {0} entries".format(len(musicNameIDMap)))
if len(musicNameIDMap) > 0:
    try:
        myMusicNameIDMap.update({k: [v[0][0], v[1].values[0]] for k,v in musicNameIDMap.items()})
    except:
        try:
            myMusicNameIDMap.update({k: [v[0][0], v[1]] for k,v in musicNameIDMap.items()})
        except:
            raise ValueError("Not sure about {0}".format(musicNameIDMap))
print("Found {0} entries".format(len(musicNameIDMap)))
print("There are {0} total entries".format(len(myMusicNameIDMap)))

Found 1 entries
Found 1 entries
There are 4692 total entries


# Save Everything

In [21]:
from ioUtils import saveFile
print("Saving {0} entries".format(len(myMusicNameIDMap)))
saveFile(ifile="musicDiscogsMap.p", idata=myMusicNameIDMap, debug=True)

Saving 4692 entries
Saving data to musicDiscogsMap.p
  --> This file is 129.3kB.
Saved data to musicDiscogsMap.p
  --> This file is 129.3kB.


In [22]:
from ioUtils import saveFile
print("Saving {0} entries".format(len(myMusicNameIDMap)))
saveFile(ifile="musicDiscogsMap.yaml", idata=myMusicNameIDMap, debug=True)

Saving 4692 entries
Saving data to musicDiscogsMap.yaml
Saved data to musicDiscogsMap.yaml
  --> This file is 188.5kB.


In [ ]:
myMusicNameIDMap = getFile("musicDiscogsMap.p")
print("Found {0} music <-> discogs maps".format(len(myMusicNameIDMap)))

In [ ]:
from ioUtils import saveFile
saveFile(ifile="musicDiscogsMap.p", idata=myMusicNameIDMap, debug=True)
from ioUtils import saveFile
saveFile(ifile="musicDiscogsMap.yaml", idata=myMusicNameIDMap, debug=True)

# Find Music and Match Albums

In [28]:
#myMusicAlbumIDMap = getFile("musicDiscogsAlbumMap.p")
myMusicAlbumIDMap = {}
print("Found {0} music <-> discogs albums maps".format(len(myMusicAlbumIDMap)))

Found 0 music <-> discogs albums maps


In [25]:
skipMusicAlbumIDMap = getFile(ifile="skipMusicDiscogsAlbumMap.yaml")
skipMusicAlbumIDMap = {}
print("Found {0} music <-> discogs albums maps".format(len(skipMusicAlbumIDMap)))

/anaconda3/envs/py37/lib/python3.7/site-packages/utils-0.0.1-py3.7.egg/ioUtils.py:72: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.


Found 0 music <-> discogs albums maps


## Johnny Cash

In [ ]:
#myMusicNameIDMap

# Show and Find Music <-> Discog Matches

In [23]:
ratioCut = 0.90
keepSkips = False

In [31]:
def showArtist(artistName, discogsArtist):
    print("\t{0: <30} ---> {1}".format(artistName, discogsArtist))
    
def showAlbum(myAlbumName):
    print("\t\tMy Album: {0}".format(myAlbumName))
    
def showMatch(myAlbumName, albumData):
    print("\t\t   Match: {0}".format(albumData))

musicAlbumIDMap = {}
discogMediaNames = ['Albums', 'Singles & EPs', 'Compilations', 'Videos', 'Miscellaneous', 'Visual', 'DJ Mixes']
myMediaNames     = ['Random', 'Todo', 'Match', 'Title', 'Singles', 'Album', 'Unknown', 'Bootleg', 'Mix', 'Tribute']
ignoreDirs       = list(set(discogMediaNames+myMediaNames))
nSkips = len(skipMusicAlbumIDMap)



for dirN in findDirs("/Users/tgadfort/matched"):
    print(dirN)
    for dirval in findDirs(dirN):        
        artistName = getDirBasics(dirval)[-1]
        artistName = normalize('NFC', artistName)
        
        discogsArtist = myMusicNameIDMap[artistName]        
        idx                      = discogsArtist[0]
        discogsArtistName        = discogsArtist[1]
        try:
            discogsArtistAlbumsData  = getRowByIndex(artistAlbumsDB, idx)
            discogsArtistAlbums      = discogsArtistAlbumsData["Albums"]
        except:
            raise ValueError("Could not find Artist ID [{0}] and Artist Name [{1}]".format(idx, discogsArtistName))
        
        albums = [x for x in findDirs(dirval) if getDirBasics(x)[-1] not in ignoreDirs]        
        myAlbumNames = [album.split("/{0}/".format(artistName))[-1] for album in albums]
        
        for myAlbumName in myAlbumNames:
            myAlbumName = normalize('NFC', myAlbumName)
            if myMusicAlbumIDMap.get(artistName) is not None:
                if myMusicAlbumIDMap[artistName].get(myAlbumName) is not None:
                    #print("\t\tMy Album: {0} ---> {1}".format(myAlbumName, myMusicAlbumIDMap[artistName][myAlbumName]))
                    continue
            if skipMusicAlbumIDMap.get(artistName) is not None:
                if skipMusicAlbumIDMap[artistName].get(myAlbumName) is not None:
                    #print("\t\tMy Album: {0} ---> {1}".format(myAlbumName, myMusicAlbumIDMap[artistName][myAlbumName]))
                    continue

            maxRatio  = None
            albumData = None
            for albumType, albumTypeData in discogsArtistAlbums.items():
                for albumID, dbAlbumName in albumTypeData.items():
                    convDiscogsAlbumName = discConv(dbAlbumName)
                    s = SequenceMatcher(None, myAlbumName, convDiscogsAlbumName)
                    ratio = round(s.ratio(), 3)
                    if ratio > ratioCut:
                        if maxRatio is None:
                            maxRatio = ratio
                        else:
                            if ratio < maxRatio:
                                continue
                            maxRatio  = ratio
                            albumData = [albumID, dbAlbumName]
                            
            if albumData is not None:
                if musicAlbumIDMap.get(artistName) is None:
                    musicAlbumIDMap[artistName] = {}
                    showArtist(artistName, discogsArtist)
                showAlbum(myAlbumName)
                musicAlbumIDMap[artistName][myAlbumName] = albumData
                showMatch(myAlbumName, albumData)
                        
            if keepSkips:
                if skipMusicAlbumIDMap.get(artistName) is None:
                    skipMusicAlbumIDMap[artistName] = {}
                #showAlbum(myAlbumName)
                skipMusicAlbumIDMap[artistName][myAlbumName] = [albumID, dbAlbumName]



/Users/tgadfort/matched/A
	Aqua                           ---> ['38862', 'Aqua']
		My Album: Megalomania
		   Match: ['3155890', 'Megalomania']
/Users/tgadfort/matched/B
	Billy Joel                     ---> ['137418', 'Billy Joel']
		My Album: Glass Houses
		   Match: ['9920037', 'Glass Houses']
		My Album: The Nylon Curtain
		   Match: ['10648173', 'The Nylon Curtain']
/Users/tgadfort/matched/C
	Carrie Underwood               ---> ['1011680', 'Carrie Underwood']
		My Album: Cry Pretty
		   Match: ['1374864', 'Cry Pretty']
	Christopher Cross              ---> ['48512', 'Christopher Cross']
		My Album: Window
		   Match: ['13322939', 'Window']
	Cyndi Lauper                   ---> ['29718', 'Cyndi Lauper']
		My Album: At Last
		   Match: ['7491780', 'At Last']
/Users/tgadfort/matched/D
	Daft Punk                      ---> ['1289', 'Daft Punk']
		My Album: One More Time
		   Match: ['14603404', 'One More Time']
	David M. Bailey                ---> ['4929306', 'David M. Bailey']
		My Album

KeyError: '300089'

## Check For Skips

In [ ]:
discogsArtist            = myMusicNameIDMap["John Fahey"]
idx                      = discogsArtist[0]
print("Artist ID -->",idx)
discogsArtistName        = discogsArtist[1]
discogsArtistAlbumsData  = getRowByIndex(artistAlbumsDB, idx)
discogsArtistAlbums      = discogsArtistAlbumsData["Albums"]

for k,v in discogsArtistAlbums.items():
    for code,name in v.items():
        print("{0: <15}{1: <12}{2}".format(k,code,name))
        continue

In [ ]:
print("Previous {0} music <-> discogs albums maps".format(nSkips))
print("Found {0} music <-> discogs albums maps".format(len(skipMusicAlbumIDMap)))
saveFile(ifile="skipMusicDiscogsAlbumMap.yaml", idata=skipMusicAlbumIDMap, debug=True)


In [ ]:
skipMusicAlbumIDMap = getFile(ifile="skipMusicDiscogsAlbumMap.yaml")
skipMusicAlbumIDMap = {}
print("Found {0} music <-> discogs albums maps".format(len(skipMusicAlbumIDMap)))

## Check For Match

In [ ]:
from ioUtils import saveFile
saveFile(ifile="newMusicDiscogsAlbumMap.yaml", idata=musicAlbumIDMap, debug=True)
print("Found {0} music <-> discogs albums maps".format(len(musicAlbumIDMap)))

In [ ]:
musicAlbumIDMap = getFile(ifile="newMusicDiscogsAlbumMap.yaml", debug=True)
print("Found {0} music <-> discogs albums maps".format(len(musicAlbumIDMap)))

## Move To Matched

In [ ]:
from fsUtils import mkDir, isDir, moveDir
for dirN in findDirs("/Users/tgadfort/matched"):
    print(dirN)
    for dirval in findDirs(dirN):        
        artistName = getDirBasics(dirval)[-1]
        artistName = normalize('NFC', artistName)
        musicMap   = musicAlbumIDMap.get(artistName)
        if musicMap is None:
            continue
        for myAlbumName,albumVal in musicMap.items():
            matchedDir = setDir(dirval, "Match")
            mkDir(matchedDir)
            srcDir = setDir(dirval, myAlbumName)
            if not isDir(srcDir):
                print("{0} does not exist".format(srcDir))
                continue
            dstDir = setDir(matchedDir, discConv(myAlbumName))
            if isDir(dstDir):
                print("{0} already exists".format(dstDir))
                continue
                
            print("Moving {0}  --->  {1}".format(srcDir, dstDir))
            moveDir(srcDir, dstDir)

# Move To Album Type

In [ ]:
moveData = {}
for dirN in findDirs("/Users/tgadfort/matched"):
    print(dirN)
    for dirval in findDirs(dirN):
        matchedDir = setDir(dirval, "Match")
        aTypes     = ['Albums', 'Singles & EPs', 'Compilations', 'Videos', 'Miscellaneous', 'Visual', 'DJ Mixes']
        albums     = [getDirBasics(x)[-1] for x in findDirs(matchedDir)]
        albums     = [x for x in albums if x not in aTypes]
        if len(albums) == 0:
            continue
        
        artistName = getDirBasics(dirval)[-1]
        artistName = normalize('NFC', artistName)
        
        discogsArtist = myMusicNameIDMap[artistName]
        
        idx                      = discogsArtist[0]
        discogsArtistName        = discogsArtist[1]
        discogsArtistAlbumsData  = getRowByIndex(artistAlbumsDB, idx)
        discogsArtistAlbums      = discogsArtistAlbumsData["Albums"]
        
        
        for myAlbumName in albums:
            myAlbumName = normalize('NFC', myAlbumName)
            maxRatio    = None
            dbAlbumData = {}
            for albumType, albumTypeData in discogsArtistAlbums.items():
                for albumID, dbAlbumName in albumTypeData.items():
                    convDiscogsAlbumName = discConv(dbAlbumName)
                    s = SequenceMatcher(None, myAlbumName, convDiscogsAlbumName)
                    ratio = round(s.ratio(), 2)
                    if ratio > 0.0:
                        if maxRatio is None:
                            maxRatio = ratio
                        else:
                            if ratio < maxRatio:
                                continue
                            maxRatio = ratio

                        if moveData.get(artistName) is None:
                            print('  ',artistName)
                            moveData[artistName] = {"Dir": matchedDir, "Albums": {}}
                        moveData[artistName]["Albums"][myAlbumName] = [albumID, albumType, convDiscogsAlbumName]
       
            if maxRatio is not None:
                print('\t{0: <50}'.format(myAlbumName),moveData[artistName]["Albums"][myAlbumName])

In [ ]:
saveFile(ifile="matchedMoveMusicDiscogsAlbumMap.yaml", idata=moveData, debug=True)
print("Found {0} music <-> discogs albums maps".format(len(moveData)))

In [ ]:
moveData = getFile(ifile="matchedMoveMusicDiscogsAlbumMap.yaml", debug=True)
print("Found {0} music <-> discogs albums maps".format(len(moveData)))

In [ ]:
atypes = {}
for artistName,artistMoveData in moveData.items():
    dirval = artistMoveData["Dir"]
    albums = artistMoveData["Albums"]
    for myAlbumName, dbAlbumData in albums.items():
        albumID, albumType, convDiscogsAlbumName = dbAlbumData[0], dbAlbumData[1], dbAlbumData[2]
        atypes[albumType] = True
        albumTypeDir = setDir(dirval, albumType)
        mkDir(albumTypeDir)
        srcDir  = setDir(dirval, myAlbumName)
        dstName = " :: ".join([myAlbumName, "[{0}]".format(albumID)])
        dstDir  = setDir(albumTypeDir, dstName)
        print("Moving {0}".format(srcDir))
        moveDir(srcDir, dstDir)

In [ ]:
list(atypes.keys())

In [ ]:
for artistName,v in musicAlbumIDMap.items():
    print(artistName)
    for myAlbumName,v2 in v.items():
        print("\t",myAlbumName,'\t',v2)

In [ ]:
if True:
    saveFile(ifile="skipMusicDiscogsAlbumMap.yaml", idata=skipMusicAlbumIDMap, debug=True)
    print("Found {0} music <-> discogs albums maps".format(len(skipMusicAlbumIDMap)))
skipMusicAlbumIDMap

In [ ]:
saveFile(ifile="newMusicDiscogsAlbumMap.yaml", idata=musicAlbumIDMap, debug=True)
print("Found {0} music <-> discogs albums maps".format(len(musicAlbumIDMap)))

In [ ]:
musicAlbumIDMap = getFile(ifile="newMusicDiscogsAlbumMap.yaml", debug=True)
print("Found {0} music <-> discogs albums maps".format(len(musicAlbumIDMap)))

## Merge Maps (if needed)

In [ ]:
for artistName,v in musicAlbumIDMap.items():
    for myAlbumName,v2 in v.items():
        if myMusicAlbumIDMap.get(artistName) is None:
            print("Adding {0}".format(artistName))
            myMusicAlbumIDMap[artistName] = {}
        if myMusicAlbumIDMap[artistName].get(myAlbumName) is None:
            print("Adding {0}/{1} ---> {2}".format(artistName, myAlbumName, v2))
            myMusicAlbumIDMap[artistName][myAlbumName] = v2

In [ ]:
from ioUtils import saveFile
saveFile(ifile="musicDiscogsAlbumMap.p", idata=myMusicAlbumIDMap, debug=True)
print("Found {0} music <-> discogs albums maps".format(len(myMusicAlbumIDMap)))

In [ ]:
saveFile(ifile="musicDiscogsAlbumMap.yaml", idata=myMusicAlbumIDMap, debug=True)

# Rename Albums

In [ ]:
musicAlbumIDMap = getFile(ifile="musicDiscogsAlbumMap.yaml", debug=True)
print("Found {0} music <-> discogs albums maps".format(len(musicAlbumIDMap)))

In [ ]:
renames = {}
for dirN in findDirs("/Users/tgadfort/matched"):
    print(dirN)
    for dirval in findDirs(dirN):        
        artistName = getDirBasics(dirval)[-1]
        artistName = normalize('NFC', artistName)
        if musicAlbumIDMap.get(artistName) is None:
            continue
        print("    {0}".format(artistName))
        for album, albumdata in musicAlbumIDMap[artistName].items():
            print("\t[{0}]   ====>   [{1}]".format(album,albumdata[1]))
            srcAlbumName = album
            dstAlbumName = discConv(albumdata[1])
            if srcAlbumName != dstAlbumName:
                if renames.get(dirval) is None:
                    renames[dirval] = {}
                renames[dirval][srcAlbumName] = dstAlbumName

In [ ]:
saveFile(ifile="musicRenames.yaml", idata=renames, debug=True)

In [ ]:
renameData = getFile(ifile="musicRenames.yaml", debug=True)

In [ ]:
from fsUtils import isDir
for dirval,artistdata in renameData.items():
    for src,dst in artistdata.items():
        srcDir = setDir(dirval, src)
        dstDir = setDir(dirval, dst)
        print(dstDir,isDir(dstDir))
        if srcDir != dstDir and not isDir(dstDir):
            print("Moving {0} -> {1}".format(srcDir, dstDir))
            moveDir(srcDir, dstDir, debug=True)